In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import ast


In [56]:
clusters = 6  #By performing the algorithms several times
              #6 clusters gave best results

In [48]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [49]:
def f(words):
    words = ast.literal_eval(words)
    next_words = words[1:]
    double = list(zip(words, next_words))
    couple = [x + '_' + y for x, y in double]
    return couple


In [50]:
df = pd.read_csv("processed_stop.csv")
df.head()

,Unnamed: 0,summary_nr,event_date,event_desc,event_keyword,const_end_use,build_stories,nonbuild_ht,project_cost,project_type,sic_list,fatality,load_dt,abstract_text
0,0,656124,2000-01-27 12:22:00,Employee's leg injured by falling load of trusses,"['UNSECURED', 'LOAD SHIFT', 'CONSTRUCTION', 'F...",B,2.0,12.0,D,A,1522.0,False,2019-01-28 00:18:04 EST,"['standing', 'atop', 'load', 'roof', 'truss', ..."
1,1,656132,2000-02-24 14:00:00,Employee injured in fall from scaffold,"['IRON WORKER', 'WORK RULES', 'CONSTRUCTION', ...",H,2.0,35.0,E,A,1791.0,False,2019-01-28 00:18:04 EST,"['iron', 'climbed', 'basket', 'articulating', ..."
2,2,656140,2000-09-28 10:30:00,Employee injured in fall from edge of second f...,"['GUARDRAIL', 'WORK RULES', 'PERIMETER GUARDIN...",H,2.0,30.0,F,A,1791.0,False,2019-01-28 00:18:04 EST,"['welding', 'stop', 'edge', 'second', 'floor',..."
3,3,656165,2000-10-16 13:00:00,Two employees injured in fall when scaffold ov...,"['ROOF', 'WORK RULES', 'SLIP', 'CONSTRUCTION',...",A,2.0,36.0,A,C,1741.0,False,2019-01-28 00:18:04 EST,"['standing', 'wooden', 'ramp', 'leading', 'top..."
4,4,656181,2000-05-24 11:00:00,Employee injured in fall when support plank gi...,"['INSTALLING', 'WORK RULES', 'CONSTRUCTION', '...",A,2.0,15.0,B,A,1751.0,False,2019-01-28 00:18:04 EST,"['standing', 'long', 'support', 'plank', 'seco..."


In [51]:
vectorizer =TfidfVectorizer(max_df=0.9)

In [52]:
#vectorize documents only fatal incidents
n_fatal = df[df["fatality"]==True]
n_fatal["abstract_text"]=n_fatal["abstract_text"].apply(f)
n_fatal["abstract_text"]=n_fatal["abstract_text"].apply(lambda x: str(' '.join(x)))


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [53]:
x = vectorizer.fit_transform(n_fatal["abstract_text"])


In [57]:
lda = LatentDirichletAllocation(n_components=clusters,
learning_method='online',learning_offset=30.).fit(x)


In [58]:
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 5)

Topic #0: power_line overhead_power aerial_lift line_electrocuted contacted_overhead
Topic #1: carbon_monoxide struck_bulldozer struck_object methylene_chloride conveyor_belt
Topic #2: run_truck motor_grader lost_control road_grader struck_run
Topic #3: fell_ground fall_protection fell_roof heart_attack fell_ladder
Topic #4: dump_truck truck_driver struck_vehicle skid_steer pickup_truck
Topic #5: trench_box wall_collapsed working_trench track_hoe sewer_line

